In [1]:
from pymongo import MongoClient
import pandas as pd
import json

In [2]:
# conexión con  mongo
client = MongoClient("mongodb://localhost:27017/")

# conexión con la base de datos "db_companies"
db = client.companies

In [3]:
query = {"$and":[
                                {"founded_year": {"$gte": 2009}},
                                {"offices": {"$exists": True}},
                                {"offices": {"$ne": None}}, 
                                {"offices.latitude":{"$ne": None}}, 
                                {"offices.longitude":{"$ne": None}},
                                {"total_money_raised": {"$regex" : "M" }}, 
                                {"founded_year": {"$exists": True}}, 
                                {"$or": [
                                    {"category_code": "design" } ,
                                    {"category_code": "web" } , 
                                    {"category_code": "software" } , 
                                    {"category_code": "games_video" } , 
                                    {"category_code": "mobile" } , 
                                    {"category_code": "enterprise" } ,   
                                    {"category_code": "analytics" } ,
                                    {'category_code': "search"},
                                    {'category_code': "network_hosting"} ,   
                                    {"category_code": "photo_video" } ,   
                                    ]} ,
                                      
                                ]
                                }

objec = {"_id":0, "name":1, "total_money_raised":1, "founded_year":1, "offices":1,
"offices.latitude":1, "offices.longitude":1}

companies=db.companies.find(query,objec)

In [4]:
data_companies= pd.DataFrame(companies)

In [5]:
data_companies.head(3)

,founded_year,name,offices,total_money_raised
0,2012,PeekYou,"[{'latitude': 40.757929, 'longitude': -73.9855...",$1.83M
1,2010,GENWI,"[{'latitude': 33.8171, 'longitude': -111.9035}]",$7.1M
2,2013,Fixya,"[{'latitude': 37.566879, 'longitude': -122.323...",$8M


In [6]:
data_companies.shape

(39, 4)

In [7]:
data_companies['offices'][34]

[{'latitude': 48.8566667, 'longitude': 2.3509871}]

In [8]:
data_companies.dtypes

founded_year           int64
name                  object
offices               object
total_money_raised    object
dtype: object

In [9]:
coords=data_companies['offices'].to_dict()
print(coords)

{0: [{'latitude': 40.757929, 'longitude': -73.985506}], 1: [{'latitude': 33.8171, 'longitude': -111.9035}], 2: [{'latitude': 37.566879, 'longitude': -122.323895}], 3: [], 4: [{'latitude': 45.5049305, 'longitude': -73.5681635}], 5: [{'latitude': 37.446823, 'longitude': -122.161523}, {'latitude': 34.0698494, 'longitude': -118.4054182}], 6: [{'latitude': 37.5645375, 'longitude': -122.3254705}], 7: [{'latitude': 32.0554, 'longitude': 34.7595}], 8: [{'latitude': 51.51819, 'longitude': -0.208979}], 9: [{'latitude': 42.275263, 'longitude': -71.24762}], 10: [{'latitude': 53.544711, 'longitude': -113.515769}], 11: [{'latitude': 39.9052263, 'longitude': -86.0547016}], 12: [{'latitude': 40.744618, 'longitude': -73.987764}], 13: [{'latitude': 48.8566667, 'longitude': 2.3509871}], 14: [{'latitude': 37.3229727, 'longitude': -122.0385792}], 15: [{'latitude': 37.270518, 'longitude': -121.955879}], 16: [{'latitude': 37.7905538, 'longitude': -122.4041485}], 17: [{'latitude': 37.317735, 'longitude': -121

In [10]:
#data_companies["ID"] = pd.DataFrame([key for key in coords.keys()]) #esto no funciona

In [11]:
#data_companies

In [12]:
#data_companies['offices']= data_companies['ID'].map(coords) #esto tampoco funciona

In [13]:
def geopoint(x):
   for dic in x:
       longitude = dic["longitude"]
       latitude = dic["latitude"]
       return {"type": "Point", "coordinates": [longitude, latitude]}
data_companies['geoindex']= data_companies['offices'].apply(geopoint)

In [14]:
data_companies.head(5)

,founded_year,name,offices,total_money_raised,geoindex
0,2012,PeekYou,"[{'latitude': 40.757929, 'longitude': -73.9855...",$1.83M,"{'type': 'Point', 'coordinates': [-73.985506, ..."
1,2010,GENWI,"[{'latitude': 33.8171, 'longitude': -111.9035}]",$7.1M,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,2013,Fixya,"[{'latitude': 37.566879, 'longitude': -122.323...",$8M,"{'type': 'Point', 'coordinates': [-122.323895,..."
3,2013,Wamba,[],€3M,None
4,2010,Needium,"[{'latitude': 45.5049305, 'longitude': -73.568...",$1M,"{'type': 'Point', 'coordinates': [-73.5681635,..."


In [15]:
data_companies.to_json (r'D:\ironhack\datamad0819\companies_clean.json', orient="records", lines=True)